In [2]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display
from copy import deepcopy

# String Finder

In [3]:
def search_strings_in_json(directory, strings_to_find):
    matches = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                filepath = os.path.join(root, file)
                with open(filepath, 'r', encoding='utf-8') as f:
                    try:
                        content = f.read()
                        for string in strings_to_find:
                            if string in content.lower():
                                matches.append((filepath, string))
                    except Exception as e:
                        print(f"Error reading {filepath}: {e}")
    return matches

# strings_to_find = ['2307148732639320', '2090275401330376', '2090275404663709', '100056956201608']
strings_to_find=['8073719999360913']

#company info and company group:61565211886319 --> people_sets_1 & child_groups, 1910535509304367

#7775445322516960
#565159963841935
#1910351919322726
#100040716736805
# directory = '/Users/dan/Documents/local_code/test/data/woolies/workplace/organization'
# directory = '/Users/dan/Documents/local_code/test/data/woolies/workplace/groups'
# directory = 'Workplace Data Company Information/groups'
# directory = 'Workplace Data Company Information/organization'
# directory = 'Workplace Data Company Information/user_profiles'
directory = 'Workplace Data Company Information/groups'
matches = search_strings_in_json(directory, strings_to_find)

count = 0

for match in matches:
    count+=1
    print(f"Found '{match[1]}' in file: {match[0]}")
print(count)

0


# JSON to pkl converter

In [3]:
import os
import json
import pandas as pd
import itertools
import pickle

def read_json_files(root_dir):
    data = {}
    if root_dir.endswith('.json'):
        temp_key_name = root_dir.split('/')[-1].split('.')[0]
        with open(root_dir, 'r', encoding='utf-8') as f:
            try:
                json_data = json.load(f)
                data[temp_key_name] = json_data
            except json.JSONDecodeError as e:
                print(f"Error reading {root_dir}: {e}")
    else:
        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if file.endswith('.json'):
                    filepath = os.path.join(root, file)
                    with open(filepath, 'r', encoding='utf-8') as f:
                        temp_key_name = root_dir + '/' + file
                        try:
                            json_data = json.load(f)
                            data[temp_key_name] = json_data
                        except json.JSONDecodeError as e:
                            print(f"Error reading {filepath}: {e}")
    return data

def extract_company_group(data_dict):
    data_dict = list(data_dict.values())[0]
    records = []
    record_template = {(key, key): value for key, value in data_dict.items() if key != 'label_values'}
    
    

    from copy import deepcopy

    for item in data_dict['label_values']:
        # print(item)
        if not "dict" in item:
            record_template[(item['ent_field_name'], item['label'])] = item.get('value', np.nan)

    for item in data_dict['label_values']:
        if "dict" not in item: continue

        record = deepcopy(record_template)
        # print(item)
        record[("title", "title")] = item['title']
        record_copy = deepcopy(record)
        
        entry_list = item['dict']

        for entry in entry_list:
            for s_entry in entry['dict']:
                # print(s_entry)

                record_copy[(s_entry['ent_field_name'], s_entry.get('label', s_entry['ent_field_name']))] = s_entry['value']

            records.append(deepcopy(record_copy))

    column_index = pd.MultiIndex.from_tuples(list(records[0].keys()))
    df = pd.DataFrame.from_records(records, columns=column_index)

    # print(records)

    return df

# def extract_data(data, file_name = ""):
#     # if file_name.startswith('company_group'):
#     #     return extract_company_group(data)
    
#     records = []
#     columns = set()

#     def check_exists_append(record, key, value, columns):
#         if record.get(key, False):
#             if isinstance(record[key], list):
#                 record[key].append(value)
#             else:
#                 record[key] = [record[key]] + [value]
#         else:
#             record[key] = value
#             columns.add(key)

#     def process_data(item, parent_record, parent_labels):
#         new_record = 0
#         # Copy the parent record to avoid mutation
#         record = parent_record.copy()
#         labels = parent_labels.copy()

#         # Make a new entry if we're at the top level
#         for key in ['timestamp', 'media', 'fbid', 'ent_name']:
#             if key in item:
#                 new_record = 1

#         if new_record == 1:
#             record = {}.copy()

#         # Extract basic fields
#         for key in ['timestamp', 'media', 'fbid', 'ent_name']:
#             if key in item:
#                 record[key] = item[key]
#                 columns.add((key, ""))

#         # Process 'label_values'
#         label_values = item.get('label_values', [])
#         if label_values:
#             for lv in label_values:
#                 ent_field_name = lv.get('ent_field_name', '')
#                 label = lv.get('label', '')
#                 key_name = [ent_field_name, label]

#                 if 'value' in lv or 'timestamp_value' in lv:
#                     value = lv.get('value') or lv.get('timestamp_value')
#                     if labels:
#                         check_exists_append(record, tuple(labels), value, columns)

#                         # records.append(record.copy())
#                     else:
#                         check_exists_append(record, tuple(key_name), value, columns)
#                 elif 'vec' in lv:
#                     # Handle 'dict' and 'vec' recursively
#                     nested_items = lv.get('vec')
#                     if nested_items:
#                         for nested_item in nested_items:
#                             process_data({'label_values': [nested_item]}, parent_record, key_name )
#                 elif 'dict' in lv:
#                     # print(lv)
#                     for entry in lv.get('dict', {}):
#                         value = entry.get('value') or entry.get('timestamp_value')
#                         temp_label = entry.get('label', '')
#                         temp_key_name = (ent_field_name, temp_label)
#                         if labels:
#                             check_exists_append(record, tuple(labels + [temp_label]), value, columns)
#                         else:
#                             check_exists_append(record, tuple(key_name + [temp_label]), value, columns)
#         else:
#             # Handle case for generic JSON file
#             for key, value in item.items():
#                 check_exists_append(record, tuple([key]), value, columns)
#         records.append(record.copy())

#     # Iterate through a list
#     for data_value in data.values():
#         if isinstance(data_value, list):
#             for item in data_value:
#                 process_data(item, {}, [])
#         else:
#             process_data(data_value, {}, [])


#     def create_multiindex_df(data):
#         # Function to flatten dictionary and preserve both tuple elements
#         def flatten_dict(d):
#             flattened = {}
#             for k, v in d.items():
#                 if isinstance(k, tuple):
#                     # Store both parts of the tuple
#                     flattened[k] = v
#                 else:
#                     # For non-tuple keys, create a tuple with same value
#                     flattened[(k, k)] = v
#             return flattened
        
#         # Flatten all dictionaries
#         flattened_data = [flatten_dict(d) for d in data]
        
#         # Create initial DataFrame
#         df = pd.DataFrame(flattened_data)
        
#         # Get all unique column tuples
#         columns = df.columns.tolist()
        

#         # Create MultiIndex columns
#         multi_index = pd.MultiIndex.from_tuples(columns)
        
#         # Create final DataFrame with MultiIndex
#         final_df = pd.DataFrame(df.values, columns=multi_index)
        
#         return final_df
#     return create_multiindex_df(records)

def extract_data_old(data, file_name = ""):
    records = []
    columns = set()

    def check_exists_append(record, key, value, columns):
        if record.get(key, False):
            if isinstance(record[key], list):
                record[key].append(value)
            else:
                record[key] = [record[key]] + [value]
        else:
            record[key] = value
            columns.add(key)

    def process_vec_items(vec_items, record, key_name, parent_labels):
        """Helper function to process vector items recursively"""
        if vec_items:
            for nested_item in vec_items:
                process_data({'label_values': [nested_item]}, record, key_name)

    def process_dict_items(dict_items, record, ent_field_name, labels):
        """Helper function to process dictionary items recursively"""
        for entry in dict_items:
            value = entry.get('value') or entry.get('timestamp_value')
            temp_label = entry.get('label', '')
            
            # Handle nested vectors within dictionary
            if 'vec' in entry:
                nested_key_name = labels + [temp_label] if temp_label else labels + [ent_field_name]
                process_vec_items(entry['vec'], record, nested_key_name, labels)
            # Handle nested dictionaries
            elif 'dict' in entry:
                nested_labels = labels + [temp_label] if temp_label else labels + [ent_field_name]
                process_dict_items(entry['dict'], record, ent_field_name, nested_labels)
            # Handle leaf values
            else:
                if labels:
                    if temp_label:
                        check_exists_append(record, tuple(labels + [temp_label]), value, columns)
                    else:
                        check_exists_append(record, tuple(labels + [ent_field_name]), value, columns)
                else:
                    check_exists_append(record, tuple([ent_field_name, temp_label]), value, columns)

    def process_data(item, parent_record, parent_labels):
        new_record = 0
        record = parent_record.copy()
        labels = parent_labels.copy()

        # Make a new entry if we're at the top level
        for key in ['timestamp', 'media', 'fbid', 'ent_name']:
            if key in item:
                new_record = 1

        if new_record == 1:
            record = {}.copy()

        # Extract basic fields
        for key in ['timestamp', 'media', 'fbid', 'ent_name']:
            if key in item:
                record[key] = item[key]
                columns.add((key, ""))

        # Process 'label_values'
        label_values = item.get('label_values', [])
        if label_values:
            for lv in label_values:
                ent_field_name = lv.get('ent_field_name', '')
                label = lv.get('label', '')
                key_name = [ent_field_name, label]

                if 'value' in lv or 'timestamp_value' in lv:
                    value = lv.get('value') or lv.get('timestamp_value')
                    if labels:
                        check_exists_append(record, tuple(labels), value, columns)
                    else:
                        check_exists_append(record, tuple(key_name), value, columns)
                elif 'vec' in lv:
                    # Handle vector items recursively
                    process_vec_items(lv['vec'], record, key_name, labels)
                elif 'dict' in lv:
                    # Handle dictionary items recursively
                    process_dict_items(lv['dict'], record, ent_field_name, 
                                    labels if labels else key_name)

        else:
            # Handle case for generic JSON file
            for key, value in item.items():
                check_exists_append(record, tuple([key]), value, columns)

        records.append(record.copy())

    # Iterate through a list
    for data_value in data.values():
        if isinstance(data_value, list):
            for item in data_value:
                process_data(item, {}, [])
        else:
            process_data(data_value, {}, [])

    def create_multiindex_df(data):
        # Function to flatten dictionary and preserve both tuple elements
        def flatten_dict(d):
            flattened = {}
            for k, v in d.items():
                if isinstance(k, tuple):
                    # Store both parts of the tuple
                    flattened[k] = v
                else:
                    # For non-tuple keys, create a tuple with same value
                    flattened[(k, k)] = v
            return flattened
        
        # Flatten all dictionaries
        flattened_data = [flatten_dict(d) for d in data]
        
        # Create initial DataFrame
        df = pd.DataFrame(flattened_data)
        
        # Get all unique column tuples
        columns = df.columns.tolist()
        
        # Create MultiIndex columns
        multi_index = pd.MultiIndex.from_tuples(columns)
        
        # Create final DataFrame with MultiIndex
        final_df = pd.DataFrame(df.values, columns=multi_index)

        def remove_duplicates_with_lists(df):
            # Convert lists to strings for comparison
            df_comparable = df.copy()
            
            # Convert empty lists to a string representation
            df_comparable = df_comparable.applymap(lambda x: str(x) if isinstance(x, list) else x)
            
            # Drop duplicates based on all columns except the first one (assuming first is index)
            # Keep first occurrence and preserve order
            unique_indices = df_comparable.iloc[:, 1:].drop_duplicates().index
            
            # Return original dataframe with duplicates removed
            return df.iloc[unique_indices]
        
        final_df_no_duplicates = remove_duplicates_with_lists(final_df)
        
        return final_df_no_duplicates
    
    return create_multiindex_df(records)

# def extract_data(data, file_name = ""):
#     records = []
#     columns = set()

#     def check_exists_append(record, key, value, columns, force_new=False):
#         """
#         Append function that combines values into lists unless force_new is True
#         """
#         if force_new:
#             record[key] = value
#             columns.add(key)
#         else:
#             if record.get(key, False):
#                 if isinstance(record[key], list):
#                     record[key].append(value)
#                 else:
#                     record[key] = [record[key]] + [value]
#             else:
#                 record[key] = value
#                 columns.add(key)

#     def process_vec_items(vec_items, base_record, key_name, parent_labels):
#         """Helper function to process vector items recursively"""
#         if vec_items:
#             for nested_item in vec_items:
#                 process_data({'label_values': [nested_item]}, base_record.copy(), key_name)

#     def has_nested_dict(dict_items):
#         """Check if there are nested dictionaries in the dict_items"""
#         for entry in dict_items:
#             if 'dict' in entry:
#                 return True
#         return False

#     def process_dict_items(dict_items, base_record, ent_field_name, labels):
#         """Helper function to process dictionary items recursively"""
#         has_nested = has_nested_dict(dict_items)
#         current_record = base_record.copy()
        
#         for entry in dict_items:
#             value = entry.get('value') or entry.get('timestamp_value')
#             temp_label = entry.get('label', '')
#             nested_ent_field_name = entry.get('ent_field_name', '')
            
#             # If there's an ent_field_name in the entry, use it
#             current_field_name = nested_ent_field_name if nested_ent_field_name else ent_field_name
            
#             # Handle nested vectors within dictionary
#             if 'vec' in entry:
#                 nested_key_name = labels + [temp_label or current_field_name]
#                 process_vec_items(entry['vec'], current_record, nested_key_name, labels)
#             # Handle nested dictionaries
#             elif 'dict' in entry:
#                 nested_labels = labels + [temp_label or current_field_name]
#                 process_dict_items(entry['dict'], current_record, current_field_name, nested_labels)
#             # Handle leaf values
#             else:
#                 if labels:
#                     key = tuple(labels + [temp_label or current_field_name])
#                 else:
#                     key = tuple([current_field_name, temp_label]) if temp_label else tuple([current_field_name, current_field_name])
                
#                 check_exists_append(current_record, key, value, columns, force_new=has_nested)
        
#         # Only append if record has data
#         if len(current_record) > 0:
#             records.append(current_record)

#     def process_data(item, parent_record, parent_labels):
#         record = parent_record.copy()
#         labels = parent_labels.copy()

#         # Extract basic fields
#         for key in ['timestamp', 'media', 'fbid', 'ent_name']:
#             if key in item:
#                 record[key] = item[key]
#                 columns.add((key, ""))

#         # Process 'label_values'
#         label_values = item.get('label_values', [])
#         if label_values:
#             for lv in label_values:
#                 ent_field_name = lv.get('ent_field_name', '')
#                 label = lv.get('label', '')
#                 key_name = [ent_field_name, label] if label else [ent_field_name, ent_field_name]

#                 if 'value' in lv or 'timestamp_value' in lv:
#                     value = lv.get('value') or lv.get('timestamp_value')
#                     if labels:
#                         check_exists_append(record, tuple(labels), value, columns)
#                     else:
#                         check_exists_append(record, tuple(key_name), value, columns)
#                 elif 'vec' in lv:
#                     # Handle vector items recursively
#                     process_vec_items(lv['vec'], record, key_name, labels)
#                 elif 'dict' in lv:
#                     # Handle dictionary items recursively
#                     process_dict_items(lv['dict'], record, ent_field_name, 
#                                     labels if labels else key_name)

#         else:
#             # Handle case for generic JSON file
#             for key, value in item.items():
#                 check_exists_append(record, tuple([key]), value, columns)

#         # Only append if record has data and is not already included
#         if len(record) > 0 and record not in records:
#             records.append(record.copy())

#     # Iterate through data
#     if isinstance(data, dict):
#         for data_value in data.values():
#             if isinstance(data_value, list):
#                 for item in data_value:
#                     process_data(item, {}, [])
#             else:
#                 process_data(data_value, {}, [])

#     def create_multiindex_df(data):
#         if not data:
#             # Return empty DataFrame with proper structure
#             return pd.DataFrame(columns=pd.MultiIndex.from_tuples([('empty', '')]))
            
#         # Function to flatten dictionary and preserve both tuple elements
#         def flatten_dict(d):
#             flattened = {}
#             for k, v in d.items():
#                 if isinstance(k, tuple):
#                     # Store both parts of the tuple
#                     flattened[k] = v
#                 else:
#                     # For non-tuple keys, create a tuple with same value
#                     flattened[(k, k)] = v
#             return flattened
        
#         # Flatten all dictionaries
#         flattened_data = [flatten_dict(d) for d in data]
        
#         # Create initial DataFrame
#         df = pd.DataFrame(flattened_data)
        
#         if df.empty:
#             return df
            
#         # Get all unique column tuples
#         columns = df.columns.tolist()
        
#         # Create MultiIndex columns
#         multi_index = pd.MultiIndex.from_tuples(columns)
        
#         # Create final DataFrame with MultiIndex
#         final_df = pd.DataFrame(df.values, columns=multi_index)
        
#         return final_df
    
#     return create_multiindex_df(records)

def is_redundant_record(prev_record, current_record):
    """Check if the current record is redundant with the previous record"""
    if not prev_record or not current_record:
        return False
    for key, value in current_record.items():
        if not value:
            continue
        if key not in prev_record or prev_record[key] != value:
            return False
    return True

def extract_data(data, file_name = ""):
    records = []
    columns = set()

    def check_exists_append(record, key, value, columns, force_new=False, is_title=False):
        """
        Append function that combines values into lists unless force_new is True
        For titles, always overwrite instead of appending
        """
        if is_title:
            record[key] = value  # Simply overwrite for titles
            columns.add(key)
        elif force_new:
            record[key] = value
            columns.add(key)
        else:
            if record.get(key, False):
                if isinstance(record[key], list):
                    record[key].append(value)
                else:
                    record[key] = [record[key]] + [value]
            else:
                record[key] = value
                columns.add(key)

    def process_vec_items(vec_items, base_record, key_name, parent_labels):
        """Helper function to process vector items recursively"""
        if vec_items:
            for nested_item in vec_items:
                process_data({'label_values': [nested_item]}, base_record.copy(), key_name)

    def process_dict_items(dict_items, base_record, ent_field_name, labels, parent_title=None, depth=0):
        """Helper function to process dictionary items recursively"""
        # if not has_nested:
        #     has_nested = has_nested_dict(dict_items)
        current_record = base_record if depth == 0 else base_record.copy()
        
        # Add parent title to record if it exists
        if parent_title:
            title_key = tuple([ent_field_name, 'title'])
            check_exists_append(current_record, title_key, parent_title, columns, is_title=True)
        
        for entry in dict_items:
            value = entry.get('value') or entry.get('timestamp_value')
            temp_label = entry.get('label', '')
            nested_ent_field_name = entry.get('ent_field_name', '')
            
            # If there's an ent_field_name in the entry, use it
            current_field_name = nested_ent_field_name if nested_ent_field_name else ent_field_name
            
            # Handle nested vectors within dictionary
            if 'vec' in entry:
                nested_key_name = labels + [temp_label or current_field_name]
                process_vec_items(entry['vec'], current_record, nested_key_name, labels)
            # Handle nested dictionaries
            elif 'dict' in entry:
                nested_labels = labels + [temp_label or current_field_name]
                process_dict_items(entry['dict'], current_record, current_field_name, 
                                nested_labels, parent_title, depth=depth+1)
            # Handle leaf values
            else:
                if labels:
                    key = tuple(labels + [temp_label or current_field_name])
                else:
                    key = tuple([current_field_name, temp_label]) if temp_label else tuple([current_field_name, current_field_name])
                
                check_exists_append(current_record, key, value, columns, force_new=depth > 0)
        
        # Only append if record has data
        if depth > 0 and current_record not in records and dict_items:
            records.append(current_record)

    def process_data(item, parent_record, parent_labels):
        record = parent_record.copy()
        labels = parent_labels.copy()

        # Extract basic fields
        for key in ['timestamp', 'media', 'fbid', 'ent_name']:
            if key in item:
                record[key] = item[key]
                columns.add((key, ""))

        # Process 'label_values'
        label_values = item.get('label_values', [])
        if label_values:
            for lv in label_values:
                ent_field_name = lv.get('ent_field_name', '')
                label = lv.get('label', '')
                key_name = [ent_field_name, label] if label else [ent_field_name, ent_field_name]
                
                if 'value' in lv or 'timestamp_value' in lv:
                    value = lv.get('value') or lv.get('timestamp_value')
                    if labels:
                        check_exists_append(record, tuple(labels), value, columns)
                    else:
                        check_exists_append(record, tuple(key_name), value, columns)
                elif 'vec' in lv:
                    # Handle vector items recursively
                    process_vec_items(lv['vec'], record, key_name, labels)
                elif 'dict' in lv:
                    # Extract title before processing dict
                    dict_title = lv.get('title')
                    # Handle dictionary items recursively with title
                    process_dict_items(lv['dict'], record, ent_field_name, 
                                    labels if labels else key_name,
                                    dict_title)
                    # # If dict is empty but has title, create a record with just the title
                    # if not lv['dict'] and dict_title:
                    #     title_record = record.copy()
                    #     title_key = tuple([ent_field_name, 'title'])
                    #     check_exists_append(title_record, title_key, dict_title, columns, is_title=True)
                    #     records.append(title_record)
                    # continue  # Skip the final record append as it's handled in process_dict_items

        else:
            # Handle case for generic JSON file
            for key, value in item.items():
                check_exists_append(record, tuple([key]), value, columns)

        if len(records) > 0 and is_redundant_record(records[-1], record):
            return

        # Only append if record has data and is not already included
        if len(record) > 0:
            records.append(record.copy())

    # Iterate through data
    if isinstance(data, dict):
        for data_value in data.values():
            if isinstance(data_value, list):
                for item in data_value:
                    process_data(item, {}, [])
            else:
                process_data(data_value, {}, [])

    def create_multiindex_df(data):
        if not data:
            # Return empty DataFrame with proper structure
            return pd.DataFrame(columns=pd.MultiIndex.from_tuples([('empty', '')]))
            
        # Function to flatten dictionary and preserve both tuple elements
        def flatten_dict(d):
            flattened = {}
            for k, v in d.items():
                if isinstance(k, tuple):
                    # Store both parts of the tuple
                    flattened[k] = v
                else:
                    # For non-tuple keys, create a tuple with same value
                    flattened[(k, k)] = v
            return flattened
        
        # Flatten all dictionaries
        flattened_data = [flatten_dict(d) for d in data]
        
        # Create initial DataFrame
        df = pd.DataFrame(flattened_data)
        
        if df.empty:
            return df
            
        # Get all unique column tuples
        columns = df.columns.tolist()
        
        # Create MultiIndex columns
        multi_index = pd.MultiIndex.from_tuples(columns)
        
        # Create final DataFrame with MultiIndex
        final_df = pd.DataFrame(df.values, columns=multi_index)
        
        return final_df
    
    return create_multiindex_df(records)


def convert_to_pkl(root_dir):
    assert not root_dir.endswith('.json'), f"ERROR: Path must be a directory not a file {root_dir}"
    root_temp = ""
    folder_loc = -1
    new_name = "default_csv"
    
    for root, dirs, files in os.walk(root_dir):
        # On first iteration we're setting up temp variables
        if not root_temp:
            root_temp = root.split("/")
            folder_loc = len(root_temp) - 1
            new_name = root_temp[folder_loc] + "_pkl_converted"
        
        # For each file in each directory
        for file in files:
            if file.endswith('.json'):
                # Create new output path
                output_root = root.split("/")
                output_root[folder_loc] = new_name
                output_root = "/".join(output_root)
                
                # Process the file
                filepath = os.path.join(root, file)
                print(f"Processing {filepath}")
                
                temp_data = read_json_files(filepath)
                temp_records = extract_data(temp_data, file)
                
                # Create output directory if it doesn't exist
                os.makedirs(output_root, exist_ok=True)
                
                # Save each DataFrame in the dictionary
                output_filename = file.replace('.json', '.pkl')
                output_csv_filename = file.replace('.json', '.csv')
                output_path = os.path.join(output_root, output_filename)
                output_csv_path = os.path.join(output_root, output_csv_filename)
                temp_records.to_pickle(output_path, protocol=pickle.HIGHEST_PROTOCOL)
                temp_records.to_csv(output_csv_path)
                print(f"Saved {output_path}")

In [7]:

temp_data = read_json_files('Workplace Data Company Information/organization/company_group_1.json')
df = extract_data(temp_data)
df.to_csv('company_group_1.csv')


# Example use

In [ ]:



# def extract_company_info(data_dict):
#     """
#     Extracts company information from a JSON structure.
#     Long description fields are not used as keys.
#     """
#     records = []
#     # Build a template from the top-level keys (except 'label_values')
#     record_template = {(key, key): value for key, value in data_dict.items() if key != 'label_values'}
    
#     # For non-nested (simple) entries, update the template.
#     for item in data_dict.get('label_values', []):
#         if "dict" not in item and "vec" not in item:
#             record_template[(item['ent_field_name'], item.get('label', np.nan))] = item.get('value', np.nan)
    
#     # Handle nested entries (records with a "dict" key)
#     for item in data_dict.get('label_values', []):
#         if "dict" not in item or "vec" not in item:
#             continue
#         base_record = deepcopy(record_template)
#         # Check if the nested entries are grouped (e.g., group installs) by looking for a nested "dict" inside.
#         first_nested = item['dict'][0] if item['dict'] else None
#         if first_nested and isinstance(first_nested, dict) and 'dict' in first_nested:
#             # This is a group of nested records
#             for group_item in item['dict']:
#                 record = deepcopy(base_record)
#                 if 'title' in group_item:
#                     record[("title", "title")] = group_item['title']
#                 for sub_entry in group_item.get('dict', []):
#                     record[(sub_entry['ent_field_name'], sub_entry.get('label', sub_entry['ent_field_name']))] = sub_entry.get('value', np.nan)
#                 records.append(record)
#         else:
#             # Otherwise, the nested dict is at one level.
#             record = deepcopy(base_record)
#             for sub_entry in item['dict']:
#                 record[(sub_entry['ent_field_name'], sub_entry.get('label', sub_entry['ent_field_name']))] = sub_entry.get('value', np.nan)
#             records.append(record)
    
#     if records:
#         column_index = pd.MultiIndex.from_tuples(list(records[0].keys()))
#         df = pd.DataFrame.from_records(records, columns=column_index)
#     else:
#         df = pd.DataFrame()
    
#     return df


# with open('Workplace Data Company Information/organization/company_info_1.json', 'r') as f:
#     data = json.load(f)

# df = extract_company_info(data)
# df.head()


""


In [7]:
# Extract and view single file
temp_data = read_json_files('Workplace Data Company Information/groups/people_sets_1.json')
temp_records = extract_data(temp_data)
temp_records.to_csv('company_info_1.csv')
temp_records

,timestamp,media,fbid,ent_name,ExternalID,CreationMethod,Members,NameRaw,Purpose,UpdateTime
,timestamp,media,fbid,ent_name,External ID,Creation Method,List of EntScimCompanyUser IDs which related to that people set,People set name,Purpose,Update time
,NaN,NaN,NaN,NaN,NaN,NaN,FBID,NaN,NaN,NaN
0,1700774648,[],2065784710446112,EntScimCompanyGroup,None,Admin,"[100048806529329, 100029807447446, 10001166228...",NaN,NaN,NaN
1,1700774648,[],2065784710446112,EntScimCompanyGroup,None,Admin,NaN,knowledge#set#22HBwQXa9N,Knowledge audience,1700774650.0
2,1688517324,[],1972752263082691,EntScimCompanyGroup,None,Admin,100020482295927,NaN,NaN,NaN
3,1688517324,[],1972752263082691,EntScimCompanyGroup,None,Admin,NaN,EntWorkTeam#people#set#ZlwOr5CTbJ,Utility set for has people sets ent,1694617180.0
4,1686183965,[],1954940858197165,EntScimCompanyGroup,None,Admin,100078697891602,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
577,1556735630,[],836451583379437,EntScimCompanyGroup,None,Admin,NaN,No IDs,Admin,1694617179.0
578,1552619167,[],808344766190119,EntScimCompanyGroup,None,Admin,"[100034003484642, 100021941614266, 10001386155...",NaN,NaN,NaN


# Convert whole directory to pkl

In [70]:
root_dir = 'Workplace Data Company Information'
convert_to_pkl(root_dir)

Saved Workplace Data Company Information_pkl_converted/manifest.pkl
Saved Workplace Data Company Information_pkl_converted/organization/post_campaigns_1.pkl
Saved Workplace Data Company Information_pkl_converted/organization/company_contact_support_cases_1.pkl
Saved Workplace Data Company Information_pkl_converted/organization/company_group_6.pkl
Saved Workplace Data Company Information_pkl_converted/organization/promoted_post_notice_2.pkl
Saved Workplace Data Company Information_pkl_converted/organization/company_info_2.pkl
Saved Workplace Data Company Information_pkl_converted/organization/company_sso_settings_1.pkl
Saved Workplace Data Company Information_pkl_converted/organization/employee_safety_checkup_1.pkl
Saved Workplace Data Company Information_pkl_converted/organization/security_logs_1.pkl
Saved Workplace Data Company Information_pkl_converted/organization/email_domain_settings_1.pkl
Saved Workplace Data Company Information_pkl_converted/organization/company_group_7.pkl
Save

In [6]:
dict_list = [
    {'name': 'Alice', 'age': 25},
    {'name': 'Bob', 'number': [1, 2, 3]},
    {'city': 'Paris', 'rating': 4.5}
]

# This will return False even though an identical dictionary exists in the list
search_dict = {'name': 'Alice', 'age': 26}
print(search_dict in dict_list)  # False

# Even the same dictionary content at a different memory location won't match
dict_list.append({'name': 'Alice', 'age': 25})
print(search_dict in dict_list)  # Still False

False
False
